In [1]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import wikipedia as wp
import yfinance as yf 
from ta.momentum import rsi 
from ta.trend import macd

In [2]:
# Try different page names for S&P 500
try:
    html = wp.page("List of S&P 500 companies").html().encode("UTF-8")
except:
    try:
        html = wp.page("S&P 500 Index").html().encode("UTF-8")
    except:
        html = wp.page("S&P 500").html().encode("UTF-8")

# Try different table indices to find the right one
tables = pd.read_html(html)
print(f"Found {len(tables)} tables")

# Usually the main S&P 500 list is in the first table (index 0)
stocks = tables[0].set_index("Symbol")
stocks = stocks.drop("GOOG", errors='ignore')  # ignore if GOOG doesn't exist
# stocks.loc["GOOGL", "Security"] = "Alphabet Inc." if "Security" in stocks.columns else stocks.loc["GOOGL", "Name"] = "Alphabet"
stocks = stocks.rename(index={"BRK.B": "BRK-B"}, errors='ignore')
stocks.head()

Found 2 tables


,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
Symbol,,,,,,,
MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [4]:
stocks[['GICS Sector']].to_csv('../input/domain.csv')

In [10]:
fundamentals = [
	yf.Ticker(stock).info for stock in stocks.index
]
fundamentals = pd.DataFrame(fundamentals).set_index("symbol")
fundamentals.index = fundamentals.index.rename("Symbol")  # for consistency with the stocks dataframe

In [11]:
fundamentals.columns, fundamentals.shape

(Index(['address1', 'city', 'state', 'zip', 'country', 'phone', 'website',
        'industry', 'industryKey', 'industryDisp',
        ...
        'exchangeDataDelayedBy', 'averageAnalystRating', 'trailingPegRatio',
        'address2', 'displayName', 'fax', 'ipoExpectedDate', 'prevName',
        'nameChangeDate', 'industrySymbol'],
       dtype='object', length=185),
 (502, 185))

In [12]:
fundamentals.head()

,address1,city,state,zip,country,phone,website,industry,industryKey,industryDisp,...,exchangeDataDelayedBy,averageAnalystRating,trailingPegRatio,address2,displayName,fax,ipoExpectedDate,prevName,nameChangeDate,industrySymbol
Symbol,,,,,,,,,,,,,,,,,,,,,
MMM,3M Center,Saint Paul,MN,55144-1000,United States,651 733 1110,https://www.3m.com,Conglomerates,conglomerates,Conglomerates,...,0,2.1 - Buy,3.2226,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AOS,11270 West Park Place,Milwaukee,WI,53224,United States,414 359 4000,https://www.aosmith.com,Specialty Industrial Machinery,specialty-industrial-machinery,Specialty Industrial Machinery,...,0,2.6 - Hold,1.7238,Suite 170,A. O. Smith,NaN,NaN,NaN,NaN,NaN
ABT,100 Abbott Park Road,North Chicago,IL,60064-6400,United States,224 667 6100,https://www.abbott.com,Medical Devices,medical-devices,Medical Devices,...,0,1.8 - Buy,NaN,Abbott Park,NaN,NaN,NaN,NaN,NaN,NaN
ABBV,1 North Waukegan Road,North Chicago,IL,60064-6400,United States,847 932 7900,https://www.abbvie.com,Drug Manufacturers - General,drug-manufacturers-general,Drug Manufacturers - General,...,0,1.9 - Buy,0.3808,NaN,AbbVie,NaN,NaN,NaN,NaN,NaN
ACN,1 Grand Canal Square,Dublin,NaN,D02 P820,Ireland,353 1 646 2000,https://www.accenture.com,Information Technology Services,information-technology-services,Information Technology Services,...,0,2.1 - Buy,1.9591,Grand Canal Harbour,Accenture,353 1 646 2020,NaN,NaN,NaN,NaN


In [15]:
for col in fundamentals.columns:
    print(col)

address1
city
state
zip
country
phone
website
industry
industryKey
industryDisp
sector
sectorKey
sectorDisp
longBusinessSummary
fullTimeEmployees
companyOfficers
auditRisk
boardRisk
compensationRisk
shareHolderRightsRisk
overallRisk
governanceEpochDate
compensationAsOfEpochDate
irWebsite
executiveTeam
maxAge
priceHint
previousClose
open
dayLow
dayHigh
regularMarketPreviousClose
regularMarketOpen
regularMarketDayLow
regularMarketDayHigh
dividendRate
dividendYield
exDividendDate
payoutRatio
fiveYearAvgDividendYield
beta
trailingPE
forwardPE
volume
regularMarketVolume
averageVolume
averageVolume10days
averageDailyVolume10Day
bid
ask
bidSize
askSize
marketCap
fiftyTwoWeekLow
fiftyTwoWeekHigh
priceToSalesTrailing12Months
fiftyDayAverage
twoHundredDayAverage
trailingAnnualDividendRate
trailingAnnualDividendYield
currency
tradeable
enterpriseValue
profitMargins
floatShares
sharesOutstanding
sharesShort
sharesShortPriorMonth
sharesShortPreviousMonthDate
dateShortInterest
sharesPercentSharesOut

In [17]:
valuation_metrics = [
    "trailingPE",      # P/E ratio - chỉ số định giá cơ bản nhất
    "forwardPE",       # P/E dự báo - xu hướng tương lai
    "priceToBook",     # P/B ratio - định giá so với tài sản
    "enterpriseToRevenue"  # EV/Revenue - định giá doanh nghiệp
]

profitability_metrics = [
    "profitMargins",   # Tỷ lệ lợi nhuận - hiệu quả kinh doanh
    "trailingEps",     # Thu nhập/cổ phiếu - năng lực sinh lời
    "forwardEps"       # EPS dự báo - tăng trưởng thu nhập
]

risk_metrics = [
    "beta",            # Độ biến động so với thị trường
    "52WeekChange",    # Biến động giá 52 tuần
    "averageVolume"    # Thanh khoản giao dịch
]

size_dividend_metrics = [
    "marketCap",       # Vốn hóa - quy mô công ty
    "fiveYearAvgDividendYield",  # Tỷ suất cổ tức
    "payoutRatio"      # Tỷ lệ chi trả cổ tức
]

key_metrics = valuation_metrics + profitability_metrics + risk_metrics + size_dividend_metrics
fundamentals[key_metrics].to_csv('../input/fundamentals.csv')

In [21]:
stocks = (fundamentals.index).to_list()
# Remove the Stock which doesn't have relevant data
stocks.remove('BF.B')

In [22]:
stock_prices = [
	yf.Ticker(stock).history(period="15y", actions=False) for stock in stocks
]

stock_prices

[                                 Open        High         Low       Close  \
 Date                                                                        
 2010-09-07 00:00:00-04:00   44.099656   44.179135   43.458535   43.522118   
 2010-09-08 00:00:00-04:00   43.559207   44.353987   43.559207   43.845329   
 2010-09-09 00:00:00-04:00   44.465261   44.550039   43.702275   43.956604   
 2010-09-10 00:00:00-04:00   43.908909   44.544730   43.792342   44.475849   
 2010-09-13 00:00:00-04:00   44.804368   45.021609   44.618920   44.963326   
 ...                               ...         ...         ...         ...   
 2025-08-27 00:00:00-04:00  156.039993  157.220001  155.750000  156.529999   
 2025-08-28 00:00:00-04:00  157.759995  158.470001  156.580002  157.559998   
 2025-08-29 00:00:00-04:00  157.600006  158.240005  155.320007  155.529999   
 2025-09-02 00:00:00-04:00  153.949997  154.490005  151.699997  154.270004   
 2025-09-03 00:00:00-04:00  153.500000  154.089996  150.429993  

In [25]:
sp500_df = pd.concat(stock_prices, keys=stocks)
sp500_df

Open        High         Low       Close  \
    Date                                                                        
MMM 2010-09-07 00:00:00-04:00   44.099656   44.179135   43.458535   43.522118   
    2010-09-08 00:00:00-04:00   43.559207   44.353987   43.559207   43.845329   
    2010-09-09 00:00:00-04:00   44.465261   44.550039   43.702275   43.956604   
    2010-09-10 00:00:00-04:00   43.908909   44.544730   43.792342   44.475849   
    2010-09-13 00:00:00-04:00   44.804368   45.021609   44.618920   44.963326   
...                                   ...         ...         ...         ...   
ZTS 2025-08-27 00:00:00-04:00  155.160004  156.110001  154.509995  155.369995   
    2025-08-28 00:00:00-04:00  155.139999  155.350006  153.289993  154.789993   
    2025-08-29 00:00:00-04:00  154.639999  156.490005  154.070007  156.399994   
    2025-09-02 00:00:00-04:00  154.979996  155.380005  151.610001  152.880005   
    2025-09-03 00:00:00-04:00  152.339996  153.089996  151.460007  152.429993   

                                Volume  
    Date                                
MMM 2010-09-07 00:00:00-04:00  3706643  
    2010-09-08 00:00:00-04:00  3333252  
    2010-09-09 00:00:00-04:00  2956153  
    2010-09-10 00:00:00-04:00  2590895  
    2010-09-13 00:00:00-04:00  3424746  
...                                ...  
ZTS 2025-08-27 00:00:00-04:00  1931100  
    2025-08-28 00:00:00-04:00  1831500  
    2025-08-29 00:00:00-04:00  1534600  
    2025-09-02 00:00:00-04:00  2228300  
    2025-09-03 00:00:00-04:00  1664700  

[1794922 rows x 5 columns]

In [26]:
sp500_df.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [28]:
sp500_df.to_csv('../input/sp500.csv')